In [1]:
import tensorflow as tf
import numpy as np

from ACNet import ACNet
from FireFightingEnv import FFEnv

from tensorflow.python.client import device_lib

dev_list = device_lib.list_local_devices()
print(dev_list)

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# __init__(self, observation_size=11, world0=None, goals0=None, DIAGONAL_MOVEMENT=False, SIZE=(10,40), PROB=(0,.5))
# TODO: add support of size, prob, etc, to initialize FFEnv Momnent lah
# Set agengt_num=1 for simplification. How to design a network support multi-agent? FIXED,follow idea 'batch'
agent_num = 3

env = FFEnv(agent_num=agent_num)

In [ ]:
env.mapEnv.plotAll()

In [ ]:
obs = np.array(env.observe_space)

# obstacle map of agent[0]
print(obs[0, 0], "\n")
# fire map of agent[0]
print(obs[0, 1], "\n")
# agent map of agent[0]
print(obs[0, 2], "\n")
# flammable map of agent[0]
print(obs[0, 3], "\n")
# reserved water of agent[0]
print(obs[0, 4], "\n")


In [ ]:
# load data
# list & NumPy array, that's a question (re: array is the best)
inputs_raw = np.array(obs[:, :4])
inputs = []
for map in inputs_raw:
    reshaped_map = [np.array(arr) for arr in map]
    inputs.append(reshaped_map)
inputs = np.array(inputs)
print(inputs.shape)

water_res = np.array(obs[:, 4]).reshape(-1,1)
print(water_res.shape)

In [ ]:
# action size: 4 direction movements, stop, 4 direction spraying with 5 kinds of range each direction
# consider a_size = 5 first (4 direction movements + stop)
# a_size = 5+4*5
a_size = 5

trainer = tf.contrib.opt.NadamOptimizer(learning_rate=1e-4)

# __init__(self, scope, a, trainer, TRAINING, GRID_SIZE)
net = ACNet( "global", a_size, trainer, True, 11)

In [ ]:
config = tf.ConfigProto()

sess = tf.Session(config=config)

sess.run(tf.global_variables_initializer()) # setting random weights for all layers of the network

In [ ]:
rnn_state = net.state_init # (zeros(1,128), zeros(1,128))

feed_dict = {
    net.inputs: inputs,
    net.water_res: water_res,
    net.state_in[0]: rnn_state[0],
    net.state_in[1]: rnn_state[1],
}

policy = sess.run(net.policy, feed_dict = feed_dict)
print(policy)

In [ ]:
state = env.reset()
episode_rollout = []
iteration = 10

actions = []
for i in range(3):
    actions.append([[], [], [], []])

for t in range(iteration):
    # 4 parameter to describe 1 action
    # Moving direction, spraying distance, spraying direction, help bacon
    for a in actions:
        a[0] = np.random.choice([0, 1, 2, 3, 4])
        a[1] = np.random.choice([0, 1, 2, 3, 4, 5])
        a[2] = np.random.choice([0, 1, 2, 3, 4])
        a[3] = np.random.choice([0, 1])

    s1, rewards, done = env.step(actions)

    state = np.array(state)
    maps_raw = np.array(state[:, :4])
    maps = []
    for map in maps_raw:
        reshaped_map = [np.array(arr) for arr in map]
        maps.append(reshaped_map)
    maps = np.array(maps)
    
    water = np.array(state[:, 4]).reshape(-1,1)

    episode_rollout.append([maps, water, actions, rewards])
    state = s1[:]

In [ ]:
# batch_size, channel, map_size[0], map_size[1]
inputs    = np.array([item[0] for item in episode_rollout]).reshape(iteration*agent_num, 4, 11, 11)
# batch_size, water_reserved
water_res = np.array([item[1] for item in episode_rollout]).reshape(iteration*agent_num, 1)
# batch_size, action_parameters
actions   = np.array([item[2] for item in episode_rollout]).reshape(iteration*agent_num, 4)
# batch_size, reward
rewards   = np.array([item[3] for item in episode_rollout]).reshape(iteration*agent_num, 1)
print(np.shape(inputs))
print(np.shape(water_res))
print(np.shape(actions))
print(np.shape(rewards))

In [ ]:
# inputs    = rollout[:,0] # 10x3x11x11x4
# water_res = rollout[:,1] # 10x3x1
# actions   = rollout[:,2] # 10x3x4
# rewards   = rollout[:,3] # 10x3x1 FIXME!!!
# valids    = rollout[:,4] # 10x3x25 FIXME!!!


# In practice: calculate long-term cumulative discounted rewards from episode rewards here
# --> net.target_v

# In practice: calculate advantages estimates (TD-error) here
# --> net.advantages


rnn_state = net.state_init
feed_dict = {
    net.inputs:      np.stack(inputs),
    net.water_res:   np.stack(water_res),
    net.actions:     np.stack(actions),

    net.target_v:    np.stack(rewards),
    net.advantages:  np.stack(rewards),

    net.state_in[0]: rnn_state[0],
    net.state_in[1]: rnn_state[1]
}

v_l, p_l, valid_l, g_n, _ = sess.run([net.value_loss,
                                      net.policy_loss,
                                      net.grad_norms,
                                      net.apply_grads], feed_dict=feed_dict)

print(v_l, p_l, valid_l, g_n)